# El commentator

In [39]:
import tabula
import pandas as pd

pdf_path = "./Bulletins/AAZZOUZ Bilel-Trimestre 2-1G2-N-6569.pdf"
pdf_path2 = "./Bulletins/AAZZOUZ Bilel-Trimestre 3-2NDE 7-N-3597.pdf"

df = tabula.read_pdf(
    pdf_path,
     pages=1,
     stream=False,
     lattice=True,
     guess=True,
     )[0]

# Drop field name column & last NaN column 
df = df.drop([df.columns[0], df.columns[6]], axis=1) 

# Drop rows with no comment
df = df.dropna(subset=[df.columns[4]]) 
#df = df.reset_index(drop=True)

df.columns = ['eleve', 'classe', "min", "max", "commentaire"]

convert_dict = {
    'eleve': float,
    'classe': float,
    'min': float,
    'max': float,
    'commentaire': str
 }

# Pre-process columns 
df = df.apply(lambda x: x.replace(',','.', regex=True) if x.name != 'commentaire' else x, axis=1)
## Filter rows with multiple student values
df = df[~df['eleve'].str.contains('\r')]

df = df.astype(convert_dict)

# Remove returns in comments
df['commentaire'] = df['commentaire'].str.replace('\r',' ', regex=True)

df

"Les résultats sont devenus insuffisants. faute de travail. L'attention en classe est problématique et la participation inexistante. Il faut vous ressaisir !"

## Extract raw data

In [73]:
import os
import tabula
import pandas as pd

directory = './Bulletins'
files = os.listdir(directory)

col_names = ['eleve', 'classe', "min", "max", "commentaire"]

def get_datastream_from_file(path):
  df = tabula.read_pdf(
      path,
      pages=1,
      stream=False,
      lattice=True,
      guess=True,
      )[0]

  df = df.drop([df.columns[0], df.columns[6]], axis=1)
  df = df.dropna()

  df.columns = col_names

  return df


files_subset = files
files_path = list(map(lambda name : directory + '/' + name, files_subset))

result = pd.DataFrame(columns=col_names)
for file_path in files_path:
  result = result.append(get_datastream_from_file(file_path))

result.to_csv('data_raw.csv')

In [20]:
# Trouver le nombre d'appréciations contenant un mot clé 

df = pd.read_csv('data.csv')
sum(['scienc' in df.iloc[i]['Appréciation'] for i in range(0, df.shape[0])])


19

## Cleanup 

In [80]:
df = pd.read_csv('data_raw.csv')
df.head()

,Unnamed: 0,eleve,classe,min,max,commentaire
0,3,"18,29","12,12","5,13","18,29",Un anglais de qualité (tant à l'oral qu'à l'éc...
1,4,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide,..."
2,5,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,6,"15,00","14,51","11,00","19,50",Bon ensemble.
4,7,"10,60","10,87","5,47","16,71",Résultats corrects mais le comportement en cla...


### Indexing

In [81]:
# data_raw has messy indexing
df = df.drop([df.columns[0]], axis=1)
df.head()

,eleve,classe,min,max,commentaire
0,"18,29","12,12","5,13","18,29",Un anglais de qualité (tant à l'oral qu'à l'éc...
1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide,..."
2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,"10,60","10,87","5,47","16,71",Résultats corrects mais le comportement en cla...


In [82]:
df.to_csv('data_clean.csv')

### Nan values

### Types

In [83]:
df = pd.read_csv('data_clean.csv', index_col=0)
df.head()

,eleve,classe,min,max,commentaire
0,"18,29","12,12","5,13","18,29",Un anglais de qualité (tant à l'oral qu'à l'éc...
1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide,..."
2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,"10,60","10,87","5,47","16,71",Résultats corrects mais le comportement en cla...


In [61]:
df.columns = ['eleve', 'classe', "min", "max", "commentaire"]
df.head()

,eleve,classe,min,max,commentaire
0,"18,29","12,12","5,13","18,29",Un anglais de qualité (tant à l'oral qu'à l'éc...
1,"18,80","12,82","6,40","18,80","Niveau d'anglais excellent. C'est très solide,..."
2,"17,80","14,28","8,30","17,80","Un excellent début d'année, poursuivez ainsi !"
3,"15,00","14,51","11,00","19,50",Bon ensemble.
4,"10,60","10,87","5,47","16,71",Résultats corrects mais le comportement en cla...


In [63]:
df = df.apply(lambda x: x.replace(',','.', regex=True) if x.name != 'commentaire' else x, axis=1)
df.head()

,eleve,classe,min,max,commentaire
0,18.29,12.12,5.13,18.29,Un anglais de qualité (tant à l'oral qu'à l'éc...
1,18.80,12.82,6.40,18.80,Niveau d'anglais excellent. C'est très solide....
2,17.80,14.28,8.30,17.80,Un excellent début d'année. poursuivez ainsi !
3,15.00,14.51,11.00,19.50,Bon ensemble.
4,10.60,10.87,5.47,16.71,Résultats corrects mais le comportement en cla...


In [72]:

sum(df['commentaire'].str.contains('\r'))

0